<a href="https://colab.research.google.com/github/DaichiIgarashi/FNN/blob/main/FNN__%E4%BB%98%E9%8C%B2%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) セットアップ

In [ ]:
#Google Colabのセットアップ
!pip install PyDrive
!pip install -U keras 
!pip install tensorflow==2.4.1
!pip install keras==2.4.3

In [ ]:
#ライブラリをインポート
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from google.colab import auth
from google.colab import drive
from google.colab import files
from oauth2client.client import GoogleCredentials
#from google.colab.patches import cv2_imshow

import tensorflow.keras
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LeakyReLU, Dense
#from keras.preprocessing import image

#from PIL import Image
#from skimage.io import imread

from scipy.stats import pearsonr
from scipy.stats import spearmanr

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

import matplotlib
import matplotlib.pyplot as plt

import cv2
import glob
import numpy as np
import pandas as pd
import math

from tqdm import tqdm
from zipfile import ZipFile

# 2) 認証
(パスワード入力)

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
Drive = GoogleDrive(gauth)

In [ ]:
drive.mount('/content/drive')

# 3) データをダウンロード

1 Input画像はGoogle Driveへdata.zipとしてダウンロード\
2 Outputはanswer.csvとしてダウンロード

In [ ]:
#train動画のリンクの貼り付け
#https://drive.google.com/file/d/1f1XWtM0CtO7UvpHNl4Aw6MZ0sNi3M9qb/view?usp=sharing

download = Drive.CreateFile({'id': '1f1XWtM0CtO7UvpHNl4Aw6MZ0sNi3M9qb'}) #リンクのIDを貼り付け
download.GetContentFile('data.zip') #フォルダの名前
!unzip data.zip #フォルダの名前(zip解除)

#dataを置き換えるか聞いてくる
#[y]は一つずつ順に置き換えるか聞いてくる
#[n]は置き換えない
#[A]は全て置き換える
#[N]は最初の一つだけ置き換える
#[r]は名前の変更

In [ ]:
#Outputのリンクの貼り付け
# https://drive.google.com/file/d/1dyqDt5SEXgZF20FQOgtIQ3H5qS0ZkQI3/view?usp=sharing

download = Drive.CreateFile({'id': '1dyqDt5SEXgZF20FQOgtIQ3H5qS0ZkQI3'})
download.GetContentFile('data.csv')

# download = Drive.CreateFile({'id': '1xVLnttfdqvkyZH2M7ydtNQqQJmgd9h7j'})
# download.GetContentFile('test.csv')

M_answer = pd.read_csv('data.csv')
M_id = M_answer['id'].values
M_label = M_answer['label'].values

# test_answer = pd.read_csv('test.csv')
# test_id = test_answer['id'].values
# test_label = test_answer['label'].values

n_labels = len(np.unique(M_label))
M_label = np.eye(n_labels)[M_label-1]

In [ ]:
#画像を出力
data_image = []
view_data_image = []
for i in tqdm(range(M_id.shape[0])):
    image = cv2.imread('data/'+'data'+M_id[i].astype('str')+'.tif',0)
    print(image.shape)
    img = image/255
    #img = 1-img
    img = (img-np.min(img))/(np.max(img)-np.min(img))
    view_data_image.append(img)
    img = img.reshape((image.shape[0]*image.shape[1]))
    data_image.append(img)
    
    plt.imshow(view_data_image[i].squeeze(), cmap=plt.cm.gray)
    # plt.show()
    print('Label = ' + str(i+1))

img_h = image.shape[0]
img_w = image.shape[1]
M_imgs = np.array(data_image)

In [ ]:
#M_allに各要素を追加
M_all = []
for i in range(M_id.shape[0]):
  M_all_append = [M_id[i],M_label[i],M_imgs[i]]
  M_all.append(M_all_append)

# print(M_all)

In [ ]:
def comb(k):
  train_data = []
  test_data = []      
  for i in range(M_id.shape[0]):
    if k*M_id.shape[0]/10 <= i < (k+1) *M_id.shape[0]/10:
      test_data_append = [M_id[i],M_label[i],M_imgs[i]]
      test_data.append(test_data_append)
    elif (k+5)*M_id.shape[0]/10 <= i < (k+6) *M_id.shape[0]/10:
      test_data_append = [M_id[i],M_label[i],M_imgs[i]]
      test_data.append(test_data_append)
    else:
      train_data_append = [M_id[i],M_label[i],M_imgs[i]]
      train_data.append(train_data_append)

  return train_data, test_data

In [ ]:
#Preparing data for each combination
id_train = []
id_test = []
y_train = []
y_test = []
x_train = []
x_test = []

for i in range(5):
  id_train.append(np.array([el[0] for el in comb(i)[0]]))
  id_test.append(np.array([el[0] for el in comb(i)[1]]))
  # y_train.append(to_categorical([el[1] for el in comb(i)[0]]))
  # y_test.append(to_categorical([el[1] for el in comb(i)[1]]))
  y_train.append(np.array([el[1] for el in comb(i)[0]]))
  y_test.append(np.array([el[1] for el in comb(i)[1]]))
  x_train.append(np.array([el[2] for el in comb(i)[0]]))
  x_test.append(np.array([el[2] for el in comb(i)[1]]))

# 4) ニューラルネットワークの構造

In [ ]:
#モデルの構造を定義
model = Sequential()
  
#隠れ層
#Denceはニューロンの数，増やしすぎると分散しちゃう，出力層に向かってニューロンを減らしていくと良い
# model.add(Dense(5, activation = 'relu', input_shape = (img_h*img_w,))) #1層目
# model.add(Dense(10, activation = 'relu')) #2層目
# model.add(Dense(5, activation = 'relu')) #3層目

#LeakyReLUを使う場合
leakyalpha = 0.01
model.add(Dense(1, input_shape = (img_h*img_w,)))
model.add(tensorflow.keras.layers.LeakyReLU(alpha = leakyalpha))
# model.add(tensorflow.keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=None))
# model.add(tensorflow.keras.layers.ELU(alpha=1.0))

#出力層
#model.add(Dense(1)) #回帰
model.add(Dense(2, activation = 'softmax', input_shape = (img_h*img_w,))) #分類 2クラス分類は'sigmoid'，多クラス分類は'softmax'

model.summary()

In [ ]:
#モデルを保存
date = 'softmax' #保存場所

path_result = '/content/drive/My Drive/result_'+ date +'/'
if os.path.exists(path_result) == 0:
  os.mkdir(path_result)

path_model = path_result + 'model/'
if os.path.exists(path_model) == 0:
  os.mkdir(path_model)

In [ ]:
#ハイパーパラメータの設定
epoch = 10000 #学習の回数
batch = 32 #バッチ数
split = 0.2 #validation率
LR = 0.00005 #学習率
ptnc = 500 #early stoppingを何回目でやるか

#callback(early stopping)の定義
callback = tensorflow.keras.callbacks.EarlyStopping(
                     monitor='val_loss', min_delta=0.000,
                     patience=ptnc, restore_best_weights=True)


#Optimizerの定義
#sgd
opt = optimizers.SGD(learning_rate = LR, momentum = 0.9) #(推奨: LR=0.0005,Mmntm=0.9)
#adam
#opt = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #強い (推奨: LR=0.01)
#adagrad
#opt = optimizers.Adagrad(lr=LR, epsilon=None, decay=0.0) #(推奨: LR=0.0001)
#adadelta
#opt = optimizers.Adadelta(lr=LR, rho=0.95, epsilon=None, decay=0.0 #(推奨: LR=0.0001)
#adamax
#opt = optimizers.Adamax(lr=LR, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0) #(推奨: LR=0.0001)
#nadam
#opt = optimizers.Nadam(lr=LR, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004) #(推奨: LR=0.0001)
#rmsprop
#opt = optimizers.RMSprop(lr=LR, rho=0.9, epsilon=None, decay=0.0) #RNNで有用性あり

#https://keras.io/ja/optimizers/ 参照

In [ ]:
#modelをコンパイル
#保存したモデルをロード
# if os.path.exists(path_model+'my_model.h5') == 1:
#   model = load_model(path_model+'my_model.h5')

# model.compile(optimizer=opt,loss='mse') #optimizer変更 回帰
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics='acc') #分類
model.save(path_model+'my_model.h5') #モデルを保存

In [ ]:
#重みとバイアスの初期設定
weight1_initial = model.layers[0].get_weights()[0]
bias1_initial = model.layers[0].get_weights()[1]
# weight2_initial = model.layers[1].get_weights()[0]
# bias2_initial = model.layers[1].get_weights()[1]
# weight3_initial = model.layers[2].get_weights()[0]
# bias3_initial = model.layers[2].get_weights()[1]
weight_out_initial = model.layers[len(model.layers)-1].get_weights()[0]
bias_out_initial = model.layers[len(model.layers)-1].get_weights()[1]

# 5) 学習

In [ ]:
#Combination1の学習
model1 = load_model(path_model+'my_model.h5') #保存したモデルを出力
history1 = model1.fit(x_train[0], y_train[0], batch_size = batch,
                      epochs = epoch, verbose = 2, callbacks = callback, #callbecksの前に#を入れればEarlyStoppingなし
                      validation_split = split)
model1.save(path_model+'my_model1.h5') #モデルの保存
#drive/Mydrive/result_???/logからval_lossを見ることが可能

In [ ]:
#Combination2の学習
model2 = load_model(path_model+'my_model.h5') #保存したモデルを出力
history2 = model2.fit(x_train[1], y_train[1], batch_size = batch,
                      epochs = epoch, verbose = 2, callbacks = callback, #callbecksの前に#を入れればEarlyStoppingなし
                      validation_split = split)
model2.save(path_model+'my_model2.h5') #モデルの保存
#drive/Mydrive/result_???/logからval_lossを見ることが可能

In [ ]:
#Combination3の学習
model3 = load_model(path_model+'my_model.h5') #保存したモデルを出力
history3 = model3.fit(x_train[2], y_train[2], batch_size = batch,
                      epochs = epoch, verbose = 2, callbacks = callback, #callbecksの前に#を入れればEarlyStoppingなし
                      validation_split = split)
model3.save(path_model+'my_model3.h5') #モデルの保存
#drive/Mydrive/result_???/logからval_lossを見ることが可能

In [ ]:
#Combination4の学習
model4 = load_model(path_model+'my_model.h5') #保存したモデルを出力
history4 = model4.fit(x_train[3], y_train[3], batch_size = batch,
                      epochs = epoch, verbose = 2, callbacks = callback, #callbecksの前に#を入れればEarlyStoppingなし
                      validation_split = split)
model4.save(path_model+'my_model4.h5') #モデルの保存
#drive/Mydrive/result_???/logからval_lossを見ることが可能

In [ ]:
#Combination5の学習
model5 = load_model(path_model+'my_model.h5') #保存したモデルを出力
history5 = model5.fit(x_train[4], y_train[4], batch_size = batch,
                      epochs = epoch, verbose = 2, callbacks = callback, #callbecksの前に#を入れればEarlyStoppingなし
                      validation_split = split)
model5.save(path_model+'my_model5.h5') #モデルの保存
#drive/Mydrive/result_???/logからval_lossを見ることが可能

In [ ]:
#モデルをロード
model1 = load_model(path_model+'my_model1.h5')
model2 = load_model(path_model+'my_model2.h5')
model3 = load_model(path_model+'my_model3.h5')
model4 = load_model(path_model+'my_model4.h5')
model5 = load_model(path_model+'my_model5.h5')

In [ ]:
#学習したモデルをコンパイル
model_all = []
model_all.append(model1)
model_all.append(model2)
model_all.append(model3)
model_all.append(model4)
model_all.append(model5)

In [ ]:
#学習履歴を保存
path_log = path_result + 'log/'
if os.path.exists(path_log) == 0:
  os.mkdir(path_log)

for i in range(1,6):
  k = globals()['history'+str(i)]
  log = pd.DataFrame(k.history)
  log_name = path_log+'log_comb'+str(i)+'.csv'
  log.to_csv(log_name, header=True, index=False)

# 6) 損失をグラフ化

## グラフの準備

In [ ]:
from matplotlib.font_manager import FontProperties

fs = 16

font = FontProperties()
font.set_family('Serif')
# font.set_name('Times New Roman')
font.set_size(fs)

font_leg = FontProperties()
font_leg.set_family('Serif')
# font.set_name('Times New Roman')
font_leg.set_size(12)

## プロットする関数を定義

In [ ]:
#相対誤差のプロット
# u_mean = 5.35

def plot_loss_acc(comb):

  # k = globals()['history' + str(comb)]

  # train_loss = k.history['loss']
  # val_loss = k.history['val_loss']
  # train_acc = k.history['acc']
  # val_acc = k.history['val_acc']

  log_name = path_log+'log_comb'+str(i)+'.csv'
  k = pd.read_csv(log_name)

  # train_loss = k['loss']
  # val_loss = k['val_loss']
  train_acc = k['acc']
  val_acc = k['val_acc']

  train_acc = 100 * train_acc
  val_acc = 100 * val_acc

  label_train_acc = 'Comb. ' + str(comb) + ' (train.)'
  label_val_acc = 'Comb. ' + str(comb) + ' (val.)'
  
  epoch_plt = list(range(1, len(val_acc) + 1))
  
  plt.plot(epoch_plt, train_acc, label = label_train_acc)
  plt.plot(epoch_plt, val_acc, label = label_val_acc)

  # fig = plt.figure()

  # train_loss_new = (train_loss**0.5/u_mean)*100 #相対誤差
  # val_loss_new = (val_loss**0.5/u_mean)*100

  #color1 = 'tab:red'
  # ax1 = fig.add_subplot(111)
  # lns1 = ax1.plot(epoch_plt, train_acc, '-r', 
  #                 label = 'Traning accuracy')#, color=color1)
  # lns2 = ax1.plot(epoch_plt, val_acc,
  #                 label = 'Validation accuracy')#, color='tab:orange')
  plt.xticks(fontsize=fs)
  plt.yticks(fontsize=fs)

  plt.xlabel('Epoch [-]', fontproperties=font)
  plt.ylabel('Accuracy [%]', fontproperties=font)
  # plt.title('Loss vs. Epoch', fontproperties=font)
  plt.legend(loc='lower right', prop=font_leg, fancybox=False, edgecolor="black")
  
  #color2 = 'tab:blue'
  # ax2 = ax1.twinx()
  # lns3 = ax2.plot(epoch_plt, train_acc, '-b',
  #                 label = 'Training accuracy')#, color=color2)
  # lns4 = ax2.plot(epoch_plt, val_acc, '-g',
  #                 label = 'Validation accuracy')#, color=color2)
  # plt.yticks(fontsize=fs)
  
  # lns = lns1+lns2#+lns3+lns4
  # labs = [l.get_label() for l in lns]
  # leg = ax1.legend(lns, labs, loc='lower right', prop=font_leg, fancybox=False, edgecolor="black")

  #ax1.grid()
  # ax1.set_xlabel('Epoch [-]', fontproperties=font)
  # ax1.set_ylabel('Accuracy [%]', fontproperties=font)
  #ax2.set_ylabel('Accuracy [%]', fontproperties=font, rotation=-90, labelpad=15)
  #ax1.set_xlim(0,8000) #表示範囲の設定
  #ax1.set_ylim(-5,105) #表示範囲の設定
  #ax2.set_ylim(-5,105)
  
  # title_loss_acc = 'Combination ' + str(comb)
  #plt.title(title_loss_acc, fontproperties=font)

In [ ]:
#損失のプロット
def plot_loss(comb):
  # k = globals()['history' + str(comb)]
  
  # train_loss = k.history['loss']
  # val_loss = k.history['val_loss']

  log_name = path_log+'log_comb'+str(i)+'.csv'
  k = pd.read_csv(log_name)

  train_loss = k['loss']
  val_loss = k['val_loss']
  
  label_train_loss = 'Comb. ' + str(comb) + ' (train.)'
  label_val_loss = 'Comb. ' + str(comb) + ' (val.)'
  
  epoch_plt = list(range(1, len(val_loss) + 1))
  plt.plot(epoch_plt, train_loss, label = label_train_loss)
  plt.plot(epoch_plt, val_loss, label = label_val_loss)
  #plt.ylim(0.0,1)

  plt.xticks(fontsize=fs)
  plt.yticks(fontsize=fs)
  plt.xlabel('Epoch [-]', fontproperties=font)
  plt.ylabel('Error [-]', fontproperties=font)
  # plt.title('Loss vs. Epoch', fontproperties=font)
  plt.legend(loc='upper right', prop=font_leg, fancybox=False, edgecolor="black")

## プロット

In [ ]:
path_LossAcc = path_result + 'LossAcc/'
if os.path.exists(path_LossAcc) == 0:
  os.mkdir(path_LossAcc)

In [ ]:
#η-Epochグラフ
for i in range(1,6):
  plot_loss_acc(i)
  plt.savefig(path_LossAcc+'CombALL_Acc.png', dpi=300, bbox_inches='tight')
  # plt.show()
##drive/Mydrive/result_???/LossAccからグラフを見ることが可能

In [ ]:
#MSE-Epochグラフ
for i in range(1,6):
  plot_loss(i)
  plt.savefig(path_LossAcc+'CombALL_Loss.png', dpi=300, bbox_inches='tight')
  # plt.show()

# 7) テスト



In [ ]:
#Define function for test result of comb_k
#variables: prob, pred, TF, result_test, Correct, Wrong
def result_test(comb):
  k = comb - 1
  
  #Checking the prediction with answer
  TF = []
  for i in range(pred[k].shape[0]):
    if pred[k][i] == y_test[k][i,1]:
      tfpn = 'T'
    else:
      tfpn = 'F'
    if pred[k][i] == 1:
      tfpn = tfpn + 'P'
    else:
      tfpn = tfpn + 'N'
    TF.append(tfpn)
    
  TF = np.array(TF)
  
  #Saving test result on a df
  # result_test = pd.DataFrame(H_test[k][:], columns = ['H'])
  result_test = pd.DataFrame(id_test[k][:], columns = ['id'])
  # result_test['id'] = id_test[k][:]
  result_test['prob.'] = prob[k][:,1]
  result_test['pred.'] = pred[k]
  result_test['ans.'] = y_test[k][:,1].astype(int)
  result_test['T/F'] = TF
  result_test['q_out'] = q_out_test[k][:,1]
  result_test['y_pred'] = y_pred_test[k][:,1]

  result_test_name = path_test+'test_comb'+str(comb)+'.csv'
  result_test.to_csv(result_test_name, header=True, index=False)
  
  #Counting correct/wrong prediction
  TP = np.count_nonzero(TF == 'TP')
  TN = np.count_nonzero(TF == 'TN')
  Total_P = np.count_nonzero(y_test[k][:,1] == 1)
  Total_N = np.count_nonzero(y_test[k][:,1] == 0)
  Correct = TP + TN
  TP_pc = TP/Total_P*100
  TN_pc = TN/Total_N*100
  Correct_pc = Correct/TF.shape[0]*100

  TPTN.append([comb, TP, Total_P, TP_pc, TN, Total_N, TN_pc, Correct,
               TF.shape[0], Correct_pc])
  
  #Print result
  # print('Comb_',comb,':')
  # print('True Positive:',TP,'/',Total_P)
  # print('True Negative:',TN,'/',Total_N)
  # print('Total Correct:',Correct,'/',TF.shape[0])

  return result_test

In [ ]:
path_test = path_result + 'test/'

if os.path.exists(path_test) == 0:
  os.mkdir(path_test)

In [ ]:
#Compiling the probability & prediction of all combinations
prob = []
pred = []

for i in range(1,6):
  i = i - 1
  # prob.append(model_all[i].predict_proba(x_test[i]))
  # pred.append(model_all[i].predict_classes(x_test[i]))
  prob.append(model_all[i].predict(x_test[i]))
  pred.append(np.argmax(model_all[i].predict(x_test[i]), axis=-1))

# for i in range(1,6):
#   i = i - 1
#   q_out.append(-np.log((1/prob[i])-1))

In [ ]:
#重みとバイアスの抽出
weight1 = []
bias1 = []
weight_out = []
bias_out = []

for i in range(5):
  weight1.append(model_all[i].layers[0].get_weights()[0])
  bias1.append(model_all[i].layers[0].get_weights()[1])
  weight_out.append(model_all[i].layers[len(model_all[i].layers)-1].get_weights()[0])
  bias_out.append(model_all[i].layers[len(model_all[i].layers)-1].get_weights()[1])

In [ ]:
W1s_test = []
q1_test = []
s1_test = []
q_out_test = []
y_pred_test = []

def relu(x):
  return x * (x>0)
def leakyRELU(x):
  return np.where(x > 0, x, x * leakyalpha)
def sigomoid(x):
  return 1/(1+np.exp(-x))
def softmax(x):
  if (x.ndim == 1):
    x = x[None,:]    # ベクトル形状なら行列形状に変換
  # テンソル（x：行列）、軸（axis=1： 列の横方向に計算）
  return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

for i in range(5):
  W1s_test.append(np.dot(x_test[i],weight1[i]))
  q1_test.append(W1s_test[i] + bias1[i])
  s1_test.append(leakyRELU(q1_test[i]))
  q_out_test.append(np.dot(s1_test[i], weight_out[i]) + bias_out[i])
  y_pred_test.append(softmax(q_out_test[i]))
  # y_pred = (np.exp(q_out[i]) / (np.sum(np.exp(q_out[i]), axis = 1)).reshape(len(M_all),1)

In [ ]:
# #Test result for all combinations
result_test_all = []
TPTN = []#[['Cmb','TP','P','TN','N','Cor','T']]

for i in range(1,6):
  result_test_all.append(result_test(i))
  #print('\n')

In [ ]:
TPTN_df = pd.DataFrame(columns=['Comb','TP','Tot P', 'TP%','TN','Tot N', 'TN%',
                                'Cor','Tot','Cor%'])

for i in range(len(TPTN)):
  TPTN_add = pd.DataFrame(
      [TPTN[i]],columns=['Comb','TP','Tot P', 'TP%','TN','Tot N', 'TN%','Cor',
                         'Tot','Cor%'])
  TPTN_df = TPTN_df.append(TPTN_add,ignore_index=True)

TPTN_df.to_csv(path_test+'test_combAll.csv', header=True, index=False)

TPTN_df

# 8) 可視化


## 8.1) 重みとバイアスの抽出と計算

In [ ]:
W1s = []
q1 = []
s1 = []
# q2 = []
# s2 = []
# q3 = []
# s3 = []
q_out = []
y_pred = []

def relu(x):
  return x * (x>0)
def leakyRELU(x):
  return np.where(x > 0, x, x * leakyalpha)
# def ELU(x):
#     return np.where(x > 0, x, 1.0 * (math.exp(x) - 1.0))
def sigomoid(x):
  return 1/(1+np.exp(-x))
def softmax(x):
  if (x.ndim == 1):
    x = x[None,:]    # ベクトル形状なら行列形状に変換
  # テンソル（x：行列）、軸（axis=1： 列の横方向に計算）
  return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

for i in range(5):
  W1s.append(np.dot(x_train[i],weight1[i]))
  q1.append(W1s[i] + bias1[i])
  s1.append(leakyRELU(q1[i]))
  q_out.append(np.dot(s1[i], weight_out[i]) + bias_out[i])
  y_pred.append(softmax(q_out[i]))
  # y_pred = (np.exp(q_out[i]) / (np.sum(np.exp(q_out[i]), axis = 1)).reshape(len(M_all),1)

In [ ]:
for i in range(5):
  # print(i+1)
  print(y_pred[i])

In [ ]:
#各値の出力
for i in range(5):
  print('\n')
  print('comb',i+1)
  print('\n')
  print("x_train")
  print(x_train[i])
  # print("weight1[:,0]")
  # print(weight1[:,0])
  print("\n W1s")
  print(W1s[i])
  print("\n bias1")
  print(bias1[i])
  print("\n q1")
  print(q1[i])
  print("\n s1")
  print(s1[i])
  print("\n weight_out")
  print(weight_out[i])
  print("\n q_out")
  print(q_out[i])
  print("\n y_pred")
  print(y_pred[i])
  print("\n bias_out")
  print(bias_out[i])

In [ ]:
#計算過程の保存

path_process = path_result + 'process/'
if os.path.exists(path_process) == 0:
  os.mkdir(path_process)

for i in range(5):
  path_comb = path_process + 'comb' + str(i+1) + '/'
  if os.path.exists(path_comb) == 0:
    os.mkdir(path_comb)
  pro_weight1 = pd.DataFrame(weight1[i])
  pro_weight1.to_csv(path_comb+'weight1_comb'+str(i+1)+'.csv', header=True, index=False)
  pro_W1s = pd.DataFrame(W1s[i])
  pro_W1s.to_csv(path_comb+'W1s_comb'+str(i+1)+'.csv', header=True, index=False)
  pro_bias1 = pd.DataFrame(bias1[i])
  pro_bias1.to_csv(path_comb+'bias1_comb'+str(i+1)+'.csv', header=True, index=False)
  pro_q1 = pd.DataFrame(q1[i])
  pro_q1.to_csv(path_comb+'q1_comb'+str(i+1)+'.csv', header=True, index=False)
  pro_s1 = pd.DataFrame(s1[i])
  pro_s1.to_csv(path_comb+'s1_comb'+str(i+1)+'.csv', header=True, index=False)
  pro_weight_out = pd.DataFrame(weight_out[i])
  pro_weight_out.to_csv(path_comb+'weight_out_comb'+str(i+1)+'.csv', header=True, index=False)
  pro_q_out = pd.DataFrame(q_out[i])
  pro_q_out.to_csv(path_comb+'q_out_comb'+str(i+1)+'.csv', header=True, index=False)
  pro_y_pred = pd.DataFrame(y_pred[i])
  pro_y_pred.to_csv(path_comb+'y_pred_comb'+str(i+1)+'.csv', header=True, index=False)
  pro_bias_out = pd.DataFrame(bias_out[i])
  pro_bias_out.to_csv(path_comb+'bias_out_comb'+str(i+1)+'.csv', header=True, index=False)

In [ ]:
#解答の保存

path_predict = path_result + 'predict/'
if os.path.exists(path_predict) == 0:
  os.mkdir(path_predict)

for i in range(5):
  predict = pd.DataFrame(np.argmax(model_all[i].predict(x_train[i]), axis=-1)+1)
  predict.to_csv(path_predict+'predict_comb'+str(i+1)+'.csv', header=True, index=False)

## 8.2) 重みの可視化

### 初期の重みの可視化

In [ ]:
#初期の重みの可視化
path_w1_ini = path_result+'w1_initial/'
if os.path.exists(path_w1_ini) == 0:
  os.mkdir(path_w1_ini)

w_max = np.max(weight1_initial)
#w_min = - w_max
w_min = np.min(weight1_initial)

for i in range(weight1_initial.shape[1]):
  print(i+1)
  f = weight1_initial[:,i]
  f_im = f.reshape((img_h, img_w))
  matplotlib.image.imsave(path_w1_ini+'w1_ini_n%d.png'%(i+1),
                          f_im, vmin=w_min, vmax=w_max, cmap='jet')
  
  fig = plt.figure()
  im = plt.imshow(f_im,'jet',vmin=w_min,vmax=w_max)
  fig.colorbar(im) #orientation='horizontal'

  plt.show()
#drive/Mydrive/result_???/w1_initialから可視化の様子を見ることが可能

### 学習後の重みの可視化
(ここで可視化)

In [ ]:
#重みの最大値と最小値を出力(この値によって可視化する重みの範囲を決定する)
for i in range(5):
  print(np.min(weight1[i]))
  print(np.max(weight1[i]))

In [ ]:
#Visualization of the weight
#variables: scale, w_max, w_min
#loops: i, f, f_im, im
def view_weight1(comb):
  k = comb - 1
  
  scale = 1.0
  # if np.max(weight1[k]) < -np.min(weight1[k]):
  #   w_max = np.max(weight1[k]) * scale
  # else:
  #   w_max = (- np.min(weight1[k])) * scale
  w_max = 0.03
  w_min = - w_max
  
#   path_w1 = path_result+'w1_trained/'
  path_w1 = path_result+'w1_trained/'
  if os.path.exists(path_w1) == 0:
    os.mkdir(path_w1)

  for i in range(weight1[k].shape[1]):
    print(i+1)
    path_w1_comb = path_w1+'w1_comb'+str(comb)+'/'
    
    if os.path.exists(path_w1_comb) == 0:
      os.mkdir(path_w1_comb)
    
    f = weight1[k][:,i]
    f_im = f.reshape((img_h, img_w))
    #f_im = (f_im - w_min)/(w_max - w_min) * 255
    matplotlib.image.imsave(path_w1_comb+'w1_comb'+str(comb)+'_n'+str(i+1)+'.png',
                            f_im, cmap='jet', vmin=w_min, vmax=w_max)
    
    fig = plt.figure()
    im = plt.imshow(f_im,'jet',vmin=w_min,vmax=w_max)
    #im.axes.get_xaxis().set_visible(False)
    #im.axes.get_yaxis().set_visible(False)
    fig.colorbar(im)
    
    plt.show()

In [ ]:
for i in range(1,6):
  print('comb',i)
  view_weight1(i)